In [ ]:
import yfinance as yf
import pandas as pd
import requests
from enum import Enum
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

NEWS_API_KEY = "ad0b9cea9d414bb99c7ffec0f22b073f"

# Define Stock Enum
class Stock(Enum):
    APPLE = "AAPL"
    TESLA = "TSLA"
    NVIDIA = "NVDA"
    AMAZON = "AMZN"
    MICROSOFT = "MSFT"
    META = "META"
    ALIBABA = "BABA"
    INTEL = "INTC"
    HP = "HPQ"

# Initialize storage and sentiment analyzer
stock_data = {}
analyzer = SentimentIntensityAnalyzer()
news_sentiment_data = []

# Fetch historical data and news for all stocks
for stock in Stock:
    try:
        print(f"Fetching data for {stock.name} ({stock.value})...")
        ticker = yf.Ticker(stock.value)
        history = ticker.history(period="max")  # Get maximum available history

        if history.empty:
            print(f"No data available for {stock.name} ({stock.value}). Skipping...")
            continue

        stock_data[stock.name] = history
        print(f"Data for {stock.name} ({stock.value}) fetched successfully!")

        # Fetch news from NewsAPI
        print(f"Fetching news for {stock.name} ({stock.value})...")
        url = f"https://newsapi.org/v2/everything?q={stock.value}&apiKey={NEWS_API_KEY}"
        response = requests.get(url)
        data = response.json()

        if "articles" not in data or not data["articles"]:
            print(f"No news found for {stock.name} ({stock.value}). Skipping...")
            continue

        for article in data["articles"]:
            headline = article["title"]
            sentiment_score = analyzer.polarity_scores(headline)["compound"]

            news_sentiment_data.append([stock.name, headline, sentiment_score, article["url"]])

        print(f"News sentiment analysis completed for {stock.name}!")

    except Exception as e:
        print(f"Error processing {stock.name} ({stock.value}): {e}")

# Convert stock data dictionary to a single DataFrame
combined_df = pd.concat(stock_data, names=["Stock", "Date"])
combined_df.reset_index(inplace=True)

# Convert news sentiment data to DataFrame
news_df = pd.DataFrame(news_sentiment_data, columns=["Stock", "Headline", "Sentiment Score", "URL"])

# Save to CSV files
combined_df.to_csv("stocks_history.csv", index=False)
news_df.to_csv("stocks_news_sentiment.csv", index=False)

# Display first few rows
print(combined_df.head())
print(news_df.head())

print("Stock data saved to stocks_history.csv and news sentiment saved to stocks_news_sentiment.csv.")


In [ ]:
import pandas as pd

# Load stock news sentiment data
news_sentiment_df = pd.read_csv("stocks_news_sentiment.csv")

# Ensure 'Sentiment Score' is numeric
news_sentiment_df["Sentiment Score"] = pd.to_numeric(news_sentiment_df["Sentiment Score"], errors="coerce")

# Calculate the average sentiment score for each stock
average_sentiment = news_sentiment_df.groupby("Stock")["Sentiment Score"].mean()

# Convert the result to a dictionary (vector format)
average_sentiment = round(average_sentiment, 3)
sentiment_vector = average_sentiment.tonumpy()

# Print the sentiment vector
print(sentiment_vector)
